In [ ]:
import numpy

In [ ]:
def generate_datasets(directory,class_limit = 10,split_ratio = 0.8):
    #class_limit = 10 # max 37
    #output_limit = 20
    #split_ratio = 0.8 # training ratio

    trainlist = []
    trainclass = []
    validlist = []
    validclass = []

    for i, klasse in enumerate(classes[:class_limit+1]):
        output = os.listdir("{}/{}".format(directory,klasse))    
        output_limit = 200* split_ratio #countlist[i] * split_ratio 

        print(f"Class: {i} - {int(output_limit)}/{countlist[i]}\t - {classes[i]}")

        # Loop for training data
        for image_path in output[:int(output_limit+1)]:
            trainlist.append(convert("{}{}/{}".format(directory,klasse, image_path)))
            trainclass.append(klasse)

        # Loop for test data
        for image_path in output[int(output_limit):200]:
            validlist.append(convert("{}{}/{}".format(directory,klasse, image_path)))
            validclass.append(klasse)

    trainlist = np.asarray(trainlist)
    validlist = np.asarray(validlist)
    train_oh = pd.get_dummies(pd.DataFrame(trainclass)).values
    valid_oh = pd.get_dummies(pd.DataFrame(validclass)).values

    return trainlist, validlist, train_oh, valid_oh

In [ ]:
def convert(image_path, target=224):   
    image = io.imread(image_path)
    rescale_factor = target / image.shape[0],target / image.shape[1]
    scaled = rescale(image, rescale_factor, anti_aliasing=False, multichannel=True,mode='constant')[:,:,:3]
    scaled = scaled.transpose((-1, 0, 1))

    return scaled

In [135]:
import os
import numpy as np
import pandas as pd
from skimage import io
from skimage.transform import rescale


class Trainloader:
    
    def __init__(self,classfolder,croppedfolder,picturesize,batchsize,split_ratio):
        self.classdirectory = classfolder
        self.cropdirectory = croppedfolder
        self.picturesize = picturesize
        self.split_ration = split_ratio
        self.batchsize = batchsize
        self.classlist = self._getclasses()
        self.countlist = self._get_countlist()
        self.onehot = self._get_onehot()
        self.trainsplit,self.validsplit = self._train_valid_split(split_ratio)
        self.batchnumber = 0
        
    
    def _getclasses(self):
        classlist = [x[1] for x in os.walk(self.classdirectory)][0]
        classlist.sort()
        
        return classlist
    
    def _get_countlist(self):
        countlist = []

        for i, klasse in enumerate(self.classlist):
            output = os.listdir("{}{}".format(self.classdirectory,klasse))    
            path, dirs, files = next(os.walk("{}{}".format(self.classdirectory,klasse)))
            file_count = len(files)
            countlist.append(file_count)

        return countlist
    
    def _get_onehot(self):
        for i,currentClass in enumerate(self.classlist):
            pictures = os.listdir("{}/{}".format(self.classdirectory,currentClass))
            df1 = pd.DataFrame([pictures,[1 for i in range(len(pictures))]]).T
            df1.columns=['picture',currentClass]
            if i != 0:
                onehot = pd.merge(df1,onehot,on='picture',how='outer')
            else:
                onehot = df1.copy()
        
        onehot = onehot.fillna(value=0)
        onehot = onehot.set_index('picture')
        return onehot
    
    def _convert_picture(self,path):
        image = io.imread(path)
        rescale_factor = self.picturesize / image.shape[0],self.picturesize / image.shape[1]
        scaled = rescale(image, rescale_factor, anti_aliasing=False, multichannel=True,mode='constant')[:,:,:3]
        scaled = scaled.transpose((-1, 0, 1))

        return scaled
    
    def _get_pictures(self,paths):
        trainlist = []
        for path in paths:
            trainlist.append(self._convert_picture(self.cropdirectory+path))
            
        return trainlist
    
    def _train_valid_split(self,split_ratio):
        split = self.onehot.index.tolist()
        
        
        train = split[:int(len(split)*split_ratio)]
        valid = split[int(len(split)*split_ratio):]
        
        validsplit = []
        for i in range(0,len(valid),self.batchsize):
             validsplit.append(valid[i:i + self.batchsize])
                
        trainsplit = []
        for i in range(0,len(train),self.batchsize):
             trainsplit.append(train[i:i + self.batchsize])
        
        return trainsplit, validsplit
        
    
    def train_next(self):
        try:
            paths = self.testsplit[self.batchnumber]
            pictures = self._get_pictures(paths)
            targets = self.onehot.loc[paths].values
        
            self.batchnumber += 1
        
            return pictures, targets
        
        except:
            self.batchnumber = 0
            return None, None
    
    def valid_next(self):
        try:
            paths = self.validsplit[self.batchnumber]
            pictures = self._get_pictures(paths)
            targets = self.onehot.loc[paths].values
        
            self.batchnumber += 1
        
            return pictures, targets
        
        except:
            self.batchnumber = 0
            return None, None
        
    
    def print_classes(self):
        
        for i in range(len(self.classlist)):
            print(f"Class: {i} - {self.countlist[i]}\t\t - {self.classlist[i]}")
        
        

In [136]:
classfolder="./dataset_v2/root/train/classes/"
croppedfolder = "./dataset_v2/root/train/cropped/"
picturesize = 224
batchsize = 64
split_ratio = 0.8

In [137]:
trainloader = Trainloader(classfolder,croppedfolder,picturesize,batchsize,split_ratio)

In [141]:
trainloader.onehot.loc[trainloader.validsplit[1]].values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [53]:
pics,onehot = trainloader.next()

In [121]:
len(trainloader.validsplit[1])

64

In [76]:
listen = []
for i in range(0,len(trainloader.validsplit),batchsize):
     listen.append(trainloader.validsplit[i:i + batchsize])
    #trainloader.validsplit

In [119]:
len(trainloader.validsplit)//batchsize

0

In [71]:
len(trainloader.trainsplit)//batchsize

145

In [133]:
print(len(trainloader.trainsplit))
res,_ = trainloader.valid_next()
i = 0
while res != None:
    if len(res) != 64:
        print(str(trainloader.batchnumber)+"\t" +str(len(res)))
        break
    res,_ = trainloader.valid_next()
    i +=1

146
37	20


In [118]:
i

38

In [8]:
trainlist

[array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]),
 array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0

In [58]:
trainloader.onehot

,yellow,wrecked,white,van,truck,tanker,sunroof,spare_wheel,soft_shell_box,small_vehicle,...,flatbed,enclosed_cab,enclosed_box,dedicated_agricultural_vehicle,crane_truck,cement_mixer,bus,blue,black,ac_vents
picture,,,,,,,,,,,,,,,,,,,,,
13580_22212.png,1,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
19531_36847.png,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
14016_37389.png,1,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
14540_18531.png,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
17591_32892.png,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
12406_11166.png,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
17710_30859.png,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
14540_11909.png,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
15104_23345.png,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
